In [1]:
import pandas as pd
import numpy as np
import os
import shutil

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def organize_and_rename_files():
    """
    Organize analysis output files by:
    1. Renaming merged_results.csv and averaged_results.csv in YA_Data and OA_Data folders
    2. Moving them to the parent data folder with appropriate prefixes
    """
    
    # Base data folder path
    data_folder = "/Volumes/YB_Drive/NavAging_Paper/data"
    ya_folder = os.path.join(data_folder, "YA_Data")
    oa_folder = os.path.join(data_folder, "OA_Data")
    
    print("Organizing and renaming analysis output files...")
    
    # Process YA_Data folder
    ya_merged_src = os.path.join(ya_folder, "merged_results.csv")
    ya_averaged_src = os.path.join(ya_folder, "averaged_results.csv")
    ya_merged_dest = os.path.join(data_folder, "ya_merged_results.csv")
    ya_averaged_dest = os.path.join(data_folder, "ya_averaged_results.csv")
    
    if os.path.exists(ya_merged_src):
        shutil.move(ya_merged_src, ya_merged_dest)
        print(f"Moved and renamed: {ya_merged_src} -> {ya_merged_dest}")
    else:
        print(f"Warning: {ya_merged_src} not found")
    
    if os.path.exists(ya_averaged_src):
        shutil.move(ya_averaged_src, ya_averaged_dest)
        print(f"Moved and renamed: {ya_averaged_src} -> {ya_averaged_dest}")
    else:
        print(f"Warning: {ya_averaged_src} not found")
    
    # Process OA_Data folder
    oa_merged_src = os.path.join(oa_folder, "merged_results.csv")
    oa_averaged_src = os.path.join(oa_folder, "averaged_results.csv")
    oa_merged_dest = os.path.join(data_folder, "oa_merged_results.csv")
    oa_averaged_dest = os.path.join(data_folder, "oa_averaged_results.csv")
    
    if os.path.exists(oa_merged_src):
        shutil.move(oa_merged_src, oa_merged_dest)
        print(f"Moved and renamed: {oa_merged_src} -> {oa_merged_dest}")
    else:
        print(f"Warning: {oa_merged_src} not found")
    
    if os.path.exists(oa_averaged_src):
        shutil.move(oa_averaged_src, oa_averaged_dest)
        print(f"Moved and renamed: {oa_averaged_src} -> {oa_averaged_dest}")
    else:
        print(f"Warning: {oa_averaged_src} not found")
    
    print("File organization completed!")
    return oa_merged_dest, oa_averaged_dest

In [3]:
def post_analysis_cleanup():
    """
    Post-analysis cleanup script that:
    1. Organizes and renames analysis output files
    2. Removes erroneous Orientation_Time for BNC39 Pawn Shop
    3. Updates Total_Time accordingly
    4. Recalculates and updates the averaged Orientation_Time in oa_averaged_results.csv
    """
    
    # Step 1: Organize files first
    merged_csv_path, averaged_csv_path = organize_and_rename_files()
    
    print("Starting post-analysis cleanup...")
    
    # Step 2: Load the merged results file
    print("Loading merged results...")
    df_merged = pd.read_csv(merged_csv_path)
    
    # Step 3: Fix BNC39 Pawn Shop data
    print("Fixing BNC39 Pawn Shop Orientation_Time error...")
    condition = (df_merged['Participant'] == 'BNC39') & (df_merged['Block_Num'] == 1) & (df_merged['Target_Name'] == 'Pawn Shop')
    
    if condition.any():
        # Before removing Orientation_Time, calculate new Total_Time by subtracting Orientation_Time from Total_Time
        df_merged.loc[condition, 'Total_Time'] = df_merged.loc[condition, 'Total_Time'] - df_merged.loc[condition, 'Orientation_Time']
        
        # Remove the erroneous Orientation_Time value
        df_merged.loc[condition, 'Orientation_Time'] = np.nan
        
        # Save the corrected merged results
        df_merged.to_csv(merged_csv_path, index=False)
        print("Successfully removed Orientation_Time value for participant BNC39 with Target_Name 'Pawn Shop'")
    else:
        print("BNC39 Pawn Shop entry not found or already corrected")
    
    # Step 4: Calculate new average Orientation_Time for BNC39
    print("Calculating new average Orientation_Time for BNC39...")
    
    # Filter for BNC39 data and exclude NaN values
    bnc39_data = df_merged[df_merged['Participant'] == 'BNC39']
    valid_orientation_times = bnc39_data['Orientation_Time'].dropna()
    
    if len(valid_orientation_times) > 0:
        new_avg_orientation_time = valid_orientation_times.mean()
        print(f"New average Orientation_Time for BNC39: {new_avg_orientation_time:.6f}")
        
        # Step 5: Update the averaged results file
        print("Updating averaged results file...")
        df_averaged = pd.read_csv(averaged_csv_path)
        
        # Find and update BNC39's Orientation_Time in averaged results
        bnc39_condition = df_averaged['Participant'] == 'BNC39'
        
        if bnc39_condition.any():
            df_averaged.loc[bnc39_condition, 'Orientation_Time'] = new_avg_orientation_time
            
            # Save the updated averaged results
            df_averaged.to_csv(averaged_csv_path, index=False)
            print(f"Successfully updated BNC39's averaged Orientation_Time to {new_avg_orientation_time:.6f}")
        else:
            print("BNC39 not found in averaged results file")
    else:
        print("No valid Orientation_Time values found for BNC39")
    
    print("Post-analysis cleanup completed!")

In [4]:
# Run the cleanup function
if __name__ == "__main__":
    post_analysis_cleanup()

Organizing and renaming analysis output files...
File organization completed!
Starting post-analysis cleanup...
Loading merged results...
Fixing BNC39 Pawn Shop Orientation_Time error...
Successfully removed Orientation_Time value for participant BNC39 with Target_Name 'Pawn Shop'
Calculating new average Orientation_Time for BNC39...
New average Orientation_Time for BNC39: 10.034298
Updating averaged results file...
Successfully updated BNC39's averaged Orientation_Time to 10.034298
Post-analysis cleanup completed!
